# Example run: FaIR

Here we demonstrate how to run with FaIR. We run with the IPs from WG3 and demonstrate that the results in the database can be reproduced. However, other input data could be used for custom scenario runs.

For more information, see the docs associated with the FaIR model.

## Setup logging

Pyam does its own logging stuff, so we have to configure logging before that import.

In [ ]:
import logging

# Increase the level to reduce the number of log messages
LOGGING_LEVEL = logging.INFO

LOGGER = logging.getLogger("pipeline")
LOGGER.setLevel(LOGGING_LEVEL)
# have to set root logger too to get messages to come through
logging.getLogger().setLevel(LOGGING_LEVEL)

logFormatter = logging.Formatter(
    "%(asctime)s %(name)s %(threadName)s - %(levelname)s:  %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
stdoutHandler = logging.StreamHandler()
stdoutHandler.setFormatter(logFormatter)

logging.getLogger().addHandler(stdoutHandler)

## Other imports

In [ ]:
import os.path

import matplotlib.pyplot as plt
import pandas.testing as pdt
import pooch
import pyam

from climate_assessment.cli import run_workflow

## Configuration of input data

### Download configuration

The FaIR setup uses two files, because it reduces the size of the configuration input compared to using only one file.

You can download these files manually from "https://zenodo.org/record/6601980", or you can download it interactively using `pooch` as demonstrated here. 

In [ ]:
fair_data_dir = os.path.join("..", "data", "fair")
os.makedirs(fair_data_dir, exist_ok=True)

In [ ]:
fair_slim_filename = "fair-1.6.2-wg3-params-slim.json"
fair_common_filename = "fair-1.6.2-wg3-params-common.json"

**N.B.** you only need to run the two cells below this the first time, after that you can also skip and run without internet connection. Of course, pooch caches all the files so running them again won't cost you any time.

In [ ]:
fair_slim_url = (
    "https://zenodo.org/record/6601980/files/fair-1.6.2-wg3-params-slim.json?download=1"
)
fair_slim_hash = "c071ca619c0ae37a6abdeb79c0cece7b"

pooch.retrieve(
    url=fair_slim_url,
    known_hash=f"md5:{fair_slim_hash}",
    path=fair_data_dir,
    fname=fair_slim_filename,
)

In [ ]:
fair_common_url = "https://zenodo.org/record/6601980/files/fair-1.6.2-wg3-params-common.json?download=1"
fair_common_hash = "42ccaffcd3dea88edfca77da0cd5789b"

pooch.retrieve(
    url=fair_common_url,
    known_hash=f"md5:{fair_common_hash}",
    path=fair_data_dir,
    fname=fair_common_filename,
)

### Set general input arguments and options to the climate assessment workflow

The values we've set below will let you run FaIR and reproduce the AR6 WG3 results.

In [ ]:
model = "fair"
model_version = "1.6.2"
fair_extra_config = os.path.join(fair_data_dir, fair_common_filename)
probabilistic_file = os.path.join(fair_data_dir, fair_slim_filename)

# Use fewer (e.g. 10) if you just want to do a test run but note that this breaks
# the stats of the probabilistic ensemble
num_cfgs = 2237
# Set to True if you're not using the full FaIR ensemble
test_run = False
# How many scenarios do you want to run in one go?
scenario_batch_size = 20

# Where should the output be saved?
outdir = os.path.join("..", "data", "output-fair-example-notebook")

### Choose input emissions pathway file

By default, we use 2 of the AR6 IPs. You could use a different file (formatted the same way) if you wish.

In [ ]:
EMISSIONS_DATA_DIR = os.path.join("..", "tests", "test-data")
EMISSIONS_INPUT_FILE = "ar6_IPs_emissions.csv"

input_emissions_file = os.path.join(EMISSIONS_DATA_DIR, EMISSIONS_INPUT_FILE)

### Choose infiller database file

We run using the infiller database that was used in CMIP6. As a result of the licensing of the scenario data, this file has to be downloaded by hand (see documentation under "Installation", section "Infiller database"). Make sure that the variable `infilling_database_file` points to where you saved this file.

In [ ]:
infilling_database_file = os.path.join(
    "..",
    "data",
    "1652361598937-ar6_emissions_vetted_infillerdatabase_10.5281-zenodo.6390768.csv",
)

## Run the climate assessment workflow

*N.B. the log with information and some warnings will be quite long - but that is nothing to worry about!*

In [ ]:
run_workflow(
    input_emissions_file,
    outdir,
    model=model,
    model_version=model_version,
    probabilistic_file=probabilistic_file,
    fair_extra_config=fair_extra_config,
    num_cfgs=num_cfgs,
    infilling_database=infilling_database_file,
    scenario_batch_size=scenario_batch_size,
)

### Load results

*N.B The filename will likely have changed if you have run your own scenarios.*

In [ ]:
output = pyam.IamDataFrame(os.path.join(outdir, "ar6_IPs_emissions_alloutput.xlsx"))
output

### Compare with database results

These would normally need to be downloaded, but we include a set in the repository for testing. Here we check that we have reproduced the database results. Obviously, this should be skipped if you have run a custom scenario.

In [ ]:
expected_output_file = os.path.join(
    "..", "tests", "test-data", "expected-output-wg3/two_ips_climate_fair.xlsx"
)

expected_db_output = pyam.IamDataFrame(expected_output_file).timeseries()

# The database does not necessarily include all the outputs we have
test_output = output.timeseries().loc[expected_db_output.index, :]

# Check that we reproduce values
pdt.assert_frame_equal(
    test_output,
    expected_db_output,
    rtol=1e-5,
    atol=1e-6,
)

### Some basic exploration

Look at the scenario categories.

In [ ]:
output.meta["Category"]

Make a plot of median warming.

In [ ]:
ax = output.filter(variable="*|Surface Temperature (GSAT)|*|50.0th Percentile").plot(
    color="scenario"
)
plt.title("Global warming above the 1850-1900 mean")
ax.set_xlim([1995, 2100])

## Conclusion

That's it! You just ran a a full climate assessment workflow going from emissions to temperature (and more) using the functionality from the climate-assessment package, and then visualised the results.

It is also possible to run from the command line, and build more elaborate workflows. For that, please see the extended documentation.